In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
info_df = pd.read_excel('./data/info.xlsx')
supp_df = pd.read_excel('./data/supply_price.xlsx')
supp_df.head(5)

In [ ]:
# 创建日期范围
start_date = datetime(2023, 6, 24)
end_date = datetime(2023, 6, 30)
date_range = pd.date_range(start_date, end_date, freq='D')
                           # 使用布尔索引筛选数据
filtered_df = supp_df[supp_df['日期'].isin(date_range)]
filtered_df

In [ ]:
available_item_codes = filtered_df.单品编码.unique().tolist()
len(available_item_codes)

In [ ]:
def grant_kind(id_):
    my_dict = info_df.set_index('单品编码')['分类编码'].to_dict()
    return my_dict.get(id_, None)

def modify_column_and_map(to_modi):
    # 使用内置的 Pandas 函数将每个单品编码转换为大类编码
    to_modi['大类编码'] = to_modi['单品编码'].map(grant_kind)

    return to_modi

In [ ]:
import warnings
warnings.filterwarnings("ignore")
available_df = modify_column_and_map(filtered_df)

In [ ]:
available_df.reset_index(drop=True)

In [ ]:
sell_df = pd.read_excel('./data/sell.xlsx')
loss_df = pd.read_excel('./data/loss.xlsx')

In [ ]:
target_list = available_item_codes
available_History_sales_info = sell_df[sell_df['单品编码'].isin(target_list)]
available_History_sales_info.reset_index(drop=True)
available_History_sales_info.head()

In [ ]:
# 创建日期范围
start_date = datetime(2023, 6, 24)
end_date = datetime(2023, 6, 30)
date_range = pd.date_range(start_date, end_date, freq='D')
                           # 使用布尔索引筛选数据
available_Recent_sales_info = available_History_sales_info[available_History_sales_info['销售日期'].isin(date_range)]
available_Recent_sales_info.drop(columns= ['扫码销售时间'], inplace= True)
available_Recent_sales_info = available_Recent_sales_info[available_Recent_sales_info['销售类型']=='销售']
available_Recent_sales_info.drop(columns= ['销售类型'], inplace= True)
available_Recent_sales_info['销售额(元)'] = available_Recent_sales_info['销量(千克)']*available_Recent_sales_info['销售单价(元/千克)']
available_Recent_sales_info.reset_index(drop= True)

In [ ]:
for item_code in available_item_codes:
    item_sales_his = available_Recent_sales_info[available_Recent_sales_info['单品编码']==item_code]
    item_sales_his.drop(columns= ['销量(千克)','销售单价(元/千克)','是否打折销售'], inplace= True)
    item_sales_his.reset_index(drop= True)
    item_sales_his = item_sales_his.groupby(['销售日期', '单品编码']).sum()
    y_data = item_sales_his['销售额(元)'].tolist()
    print(item_sales_his.head(7))
    break

In [ ]:
from scipy.optimize import minimize, LinearConstraint, Bounds, linprog

class RegressionModel:
    def __init__(self, wholesale_price_df, sales_fore, avg_price):
        self.wholesale_price_df = wholesale_price_df ## 批发价
        self.average_price = avg_price ## 平均价格
        self.preliminary_pricing = [a*1.1 for a in self.average_price] ## 初步定价
        self.sales_forecast = sales_fore ## 预测得到的销量
        self.cost_pricing = [] ## 成本加成定价
        self.purchase_quantity = [] ## 进货量
        self.Optimal_X, self.Max_Profit = [], [] ## 约束条件下的多情况解

    def determine_cost_pricing(self):
        # 根据销售量预测确定成本加成定价
        cost_pricing = self.average_price
        thread = 30
        for sale in self.sales_forecast:
            if sale>thread: ratio = 1.2
            elif sale>thread-10:  ratio = 1.3
            else: ratio = 1.4
            cost_pricing[self.sales_forecast.index(sale)] *= ratio

        # 将计算得到的成本加成定价存入 cost_pricing 列表
        self.cost_pricing = cost_pricing

    def get_purchase_quantity(self):
        # 根据销售量预测计算进货量
        ratio = 1.1
        purchase_quantity = self.sales_forecast
        purchase_quantity = [x * ratio for x in purchase_quantity]
        self.purchase_quantity = purchase_quantity

    def optimize_solution(self):
        self.determine_cost_pricing()
        # 优化求解函数，计算利润
        Max_profit = []
        for i in range(7):
            pro = (self.cost_pricing[i] * self.sales_forecast[i]) - (self.wholesale_price_df[i] * self.purchase_quantity[i])
            Max_profit.append(pro) ## 考虑运损率-->定价：sales_forecast中已考虑了各类的运损率
        self.Max_Profit = Max_profit

    def best_purchase_quantity(self):
        return self.purchase_quantity

    def optimize_with_bounds(self, available_products_name):
        self.purchase_quantity, self.Max_Profit = [[]], [[]]

        def decrease(list, num):
            new_list = [x-num if x-num >0 else x for x in list]
            return new_list

        for n in range(27, 34):
            # 目标函数：最大化收益
            c = [[y for y in x] for x in self.cost_pricing]  # 将定价取负号，转换为最大化问题
            s = self.preliminary_pricing
            # 约束条件
            A_eq = [[1] * n]  # 可售单品总数的约束
            b_eq = [27]  # 最小可售单品总数
            A_ub = [[-1] * n]  # 单品补货量的约束
            b_ub = [-2.5]  # 最小陈列量约束

            # 可售单品总数的上限约束
            A_ub.append([1] * n)
            b_ub.append(33)

            # 陈列量的最大陈列量约束
            A_ub.append([1] * n)
            b_ub.append(8)  # 最大陈列量 33种单品 共 257.8 千克的总销量封顶(最大峰值单日销售总量) 257.8/33 ~= 7.9
            for i in range(n):
                A_ub.append([1] * n)
                b_ub.append(decrease(s, 7))  # 超过7千克的部分

                if s > 10:
                    A_ub.append([1] * n)
                    b_ub.append(decrease((s, 10)))  # 超过10千克的部分

            # 单位利润衰减的约束
            for i in range(n):
                A_ub.append([0] * n)
                b_ub.append((decrease(s, 7)) * 0.5 / s)  # 超过7千克的部分每多一千克，单位利润降低

                if s > 10:
                    A_ub.append([0] * n)
                    b_ub.append(decrease(s, 10) * 0.7)  # 超过10千克的部分，单位利润降低

            # 定价的上下限约束
            bounds = [(0, self.cost_pricing[i] * 1.6) for i in range(n)]  # 最高价格小于原价格的1.6倍

            # 求解线性规划
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

            # 解析结果
            replenishment = res.x  # 单品陈列量
            pricing = -res.fun  # 定价策略

            # 输出结果
            for i in range(n):
                print(f"单品{available_products_name[i]}：补货量={replenishment[i]}，定价={pricing[i]}")

        self.Optimal_X = replenishment
        df = pd.DataFrame(list(zip(available_products_name, pricing, replenishment)), columns=['单品名称', '单价(元/千克)', '补货量(千克)'])
        df.to_excel('7月1日单品进货定价策略.xlsx', index= False)

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta
pre = './7月预测批发价'
kind_pre = './销售量数据/单类/每日'
Kind_Name = os.listdir(kind_pre)
for i in range(6):
    print(Kind_Name[i])

In [ ]:
files = os.listdir(pre)
result_df = pd.DataFrame()
for filename in files:
    file_path = os.path.join(pre, filename)
    file = pd.read_csv(file_path)
    column_name = filename.split('_')[0]
    column_prefix = column_name
    column_data = file.iloc[:, 0]
    result_df[column_name] = column_data
result_df.to_csv('./fore.csv', index=False)

In [ ]:
pre = './价格-销售重量'
files = os.listdir(pre)
fore = pd.read_csv('fore.csv') ##批发价预测
Kind_Name = []
Price_List = []
## 根据预测结果获取预计销量
for file_name in files:
    kind_name = (''.join(file_name.split('.')[0])).split('_')[0]
    Kind_Name.append(kind_name)
    file_path = os.path.join(pre, file_name)
    file = pd.read_csv(file_path)
    kind_df = pd.DataFrame(file)
    kind_fore_sales = fore[kind_name].tolist() ##销量预计： fore
    cost_price = []
    min_price = []
    for sale in kind_fore_sales:
        base = kind_df['销售重量（kg）'].median()
        max_ = kind_df['销售重量（kg）'].max()
        min_ = kind_df['销售重量（kg）'].min()
        new_df = pd.DataFrame()
        if base == 0:
            new_df = kind_df[['价格（元）']]
        else:
            new_df = kind_df[(kind_df['销售重量（kg）'] > 0.1*sale)][['价格（元）']]

        avg_cost = new_df['价格（元）'].mean() ## 区间批发价均值
        min_cost = new_df['价格（元）'].min() ## 区间批发价格最小值
        i = 1.001
        while avg_cost==0 or min_cost==0:
            new_df = kind_df[(kind_df['销售重量（kg）'] < sale*i) & (kind_df['销售重量（kg）'] > sale*(1/i))]['价格（元）']
            i *= 1.005
            avg_cost = new_df['价格（元）'].mean()
            min_cost = new_df['价格（元）'].min()
        cost_price.append(avg_cost)
        min_price.append(min_cost)

    # 设定输入数据
    model = RegressionModel(wholesale_price_df=min_price, sales_fore=kind_fore_sales, avg_price=cost_price)

    # 调用模型内部函数
    model.determine_cost_pricing()
    model.get_purchase_quantity()

    # 调用优化求解函数
    model.optimize_solution()

    # 输出利润
    # print(f"7月1日-7日{kind_name} 每天日总收益(元): ", model.Max_Profit)
    # print(f"7月1日-7日{kind_name} 每天进货量(千克): ", model.purchase_quantity)
    # print(f"7月1日-7日{kind_name} 每天定价(元)", model.cost_pricing)
    start_date = datetime(2023, 7, 1)
    date_range = [start_date + timedelta(days=i) for i in range(7)]
    result_df = pd.DataFrame({'日期': date_range,
                   '进货量(千克)': model.purchase_quantity ,
                   '成本加成定价(元/千克)': model.cost_pricing,
                   '预计总收益(元)': model.Max_Profit})
    name = f'./Q2_result/{kind_name}_23年7月1日至7日预计.xlsx'
    result_df.to_excel(name, index= False)

In [ ]:
# Merge the datasets on "单品编码"
merged_data = pd.merge(sell_df, info_df, on="单品编码", how="left")
# Display the first few rows of the merged dataset
merged_data.head(5)

In [ ]:
# 按品类和商品分类销售量
category_sales = merged_data.groupby('分类名称')['销量(千克)'].sum().sort_values(ascending=False)
product_sales = merged_data.groupby('单品名称')['销量(千克)'].sum().sort_values(ascending=False)

In [ ]:
merged_data = merged_data[merged_data['销售类型']=='销售']
merged_data.drop(columns=['是否打折销售', '销售类型'], inplace= True)
merged_data['销售额(元)'] = merged_data['销量(千克)']*merged_data['销售单价(元/千克)']
merged_data.sort_values(by='销售额(元)', inplace= True, ascending=False)
merged_data.reset_index(drop=True)

In [ ]:
mask = ((pd.to_datetime('2023-05-30') > merged_data['销售日期']) & (merged_data['销售日期'] > pd.to_datetime('2023-02-28'))) | \
       ((pd.to_datetime('2022-05-30') > merged_data['销售日期']) & (merged_data['销售日期'] > pd.to_datetime('2022-02-28'))) | \
       ((pd.to_datetime('2021-05-30') > merged_data['销售日期']) & (merged_data['销售日期'] > pd.to_datetime('2021-02-28')))

mean_sales_df = merged_data[mask].groupby('单品名称')[['销量(千克)']].max().reset_index()

forecasted_df = mean_sales_df.copy()
forecasted_df = pd.DataFrame(forecasted_df)
forecasted_df.columns = ['预测销量_7月1日' if col == '销量(千克)' else col for col in forecasted_df.columns]
forecasted_df.head(33)
SUM = forecasted_df.预测销量_7月1日.sum()
print(SUM) ## 257.865，说明一天最大支持的能够同时售出的总质量在257，反馈用于模型建立中陈列量upper_bound

In [ ]:
pricing_data = pd.merge(supp_df, info_df, on="单品编码", how="left")
avg_wholesale_price = pricing_data.groupby('分类名称')['批发价格(元/千克)'].mean()

In [ ]:
product_cost_data = pd.merge(forecasted_df, pricing_data, on="单品名称", how="left")
product_cost_data = pd.merge(product_cost_data, loss_df, left_on='分类名称', right_on='小分类名称', how="left")

# Calculate the cost per kg considering the loss rate
product_cost_data['单位批发价'] = product_cost_data['批发价格(元/千克)'] * (1 + product_cost_data['平均损耗率(%)_小分类编码_不同值'] / 100)

product_cost_data[['单品名称', '预测销量_7月1日', '批发价格(元/千克)', '平均损耗率(%)_小分类编码_不同值', '单位批发价']].head()

def determine_product_markup_rate(sales_forecast):
    if sales_forecast < 10 :
        return 1.3
    elif sales_forecast < 20:
        return 1.2
    else: return 1.1

# Calculate the expected profit for each product
product_cost_data['markup_rate'] = product_cost_data['预测销量_7月1日'].apply(determine_product_markup_rate)
product_cost_data['单价(元/千克)'] = product_cost_data['单位批发价'] * product_cost_data['markup_rate']
product_cost_data['expected_profit_per_kg'] = product_cost_data['单价(元/千克)'] - product_cost_data['单位批发价']
product_cost_data['预计总收益(元)'] = product_cost_data['expected_profit_per_kg'] * product_cost_data['预测销量_7月1日']

# Sort the products based on the expected profit
sorted_products = product_cost_data.sort_values(by='预计总收益(元)', ascending=False).drop_duplicates(subset='单品名称')

sorted_products[['单品名称', '预测销量_7月1日', '单位批发价', '单价(元/千克)', '预计总收益(元)']].head(33)